# Propose to open an Asian resturant in canberra and where 

In [327]:
# convert an address into latitude and longitude values
from geopy.geocoders import Nominatim

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

from pandas.io.json import json_normalize

import requests

print('Libraries imported.')

Libraries imported.


In [328]:
# Discover Canberra latitude and longitude
address = 'Canberra, Australia'

geolocator = Nominatim(user_agent="to_explorer", timeout=3)
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Canberra, ON, Australia are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Canberra, ON, Australia are -35.2975906, 149.1012676.


In [329]:
# Discover City centre Canberra latitude and longitude
address_city = 'city, Canberra, Australia'

geolocator = Nominatim(user_agent="to_explorer", timeout=3)
location_city = geolocator.geocode(address_city)
latitude_city = location_city.latitude
longitude_city = location_city.longitude
print('The geograpical coordinate of Canberra, ON, Australia are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Canberra, ON, Australia are -35.2975906, 149.1012676.


# Create a map of Canberra Australia 

In [366]:
#create the Toronto map 
map_canberra = folium.Map(location=[latitude, longitude], zoom_start=10)
#print Toronto map 
map_canberra

In [331]:
#Foursquare ID
CLIENT_ID = '1DQJY4DOUFEPCGCI0403CLF24SKKCZ2R5RTMDEMZZ1OB2CCJ'
# Foursquare SECRET
CLIENT_SECRET = 'RXLBLRSD4YA0OGQTHMZKTOD2JYV4QU3N445MIJB1SP0N1MOC'
# Foursquare API version
VERSION = '20180605' 
#sanity check 
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 1DQJY4DOUFEPCGCI0403CLF24SKKCZ2R5RTMDEMZZ1OB2CCJ
CLIENT_SECRET:RXLBLRSD4YA0OGQTHMZKTOD2JYV4QU3N445MIJB1SP0N1MOC


In [332]:
LIMIT = 1000 # limit of number of venues returned by Foursquare API
radius = 1000 # define radius

In [333]:
#create the GET request URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude_city, 
    longitude_city, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=1DQJY4DOUFEPCGCI0403CLF24SKKCZ2R5RTMDEMZZ1OB2CCJ&client_secret=RXLBLRSD4YA0OGQTHMZKTOD2JYV4QU3N445MIJB1SP0N1MOC&v=20180605&ll=-35.2812678,149.1291868&radius=1000&limit=1000'

In [367]:
# Send the GET request for Json file and sanity check
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e1e8211aba297001cfeb214'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'City',
  'headerFullLocation': 'City, Canberra',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 127,
  'suggestedBounds': {'ne': {'lat': -35.27226779099999,
    'lng': 149.14019122687955},
   'sw': {'lat': -35.290267809000014, 'lng': 149.11818237312048}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '529522d3498e02ea316adfb3',
       'name': 'The Cupping Room',
       'location': {'address': '1/13 University Ave.',
        'lat': -35.280471715528144,
        'lng': 149.12610648638585,
        'labeledLatLngs': [{'label': 'display',
          'lat': -3

In [335]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# extract data from Json and import to Dataframe 
# print top 10 in dataframe 

In [336]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng','venue.location.distance','venue.location.address']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng,distance,address
0,The Cupping Room,Coffee Shop,-35.280472,149.126106,293,1/13 University Ave.
1,Mezzalira Ristorante,Italian Restaurant,-35.279308,149.127748,254,West Row
2,Guild,Gaming Cafe,-35.279644,149.130832,234,"Bailey's Corner, 150 London Cct"
3,Impact Comics,Comic Shop,-35.278405,149.132024,409,16 Garema Pl
4,Harvest,Coffee Shop,-35.278223,149.126675,408,40 Marcus Clarke St


In [370]:
#recreate the Canberra map to be used to add all venues to it 
map_canberra_venues = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, name, categories in zip(nearby_venues['lat'], nearby_venues['lng'], nearby_venues['name'], nearby_venues['categories']):
    label = '{}, {}'.format(name, categories)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_canberra_venues)  

map_canberra_venues

In [337]:
# number of rows 
nearby_venues.shape

(100, 6)

In [338]:
# print venues returned by Foursquare
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


# Group and count the number of identified the venues 

In [339]:
nearby_venues.groupby('categories').count()

,name,lat,lng,distance,address
categories,,,,,
Asian Restaurant,2,2,2,2,2
Bakery,1,1,1,1,1
Bar,2,2,2,2,2
Beer Bar,1,1,1,1,1
Bookstore,2,2,2,2,2
Brewery,1,1,1,1,1
Burger Joint,2,2,2,2,2
Burrito Place,1,1,1,1,1
Café,10,10,10,10,10


# Clean the data and extract only Asian restaurants 
# print the Asian data frame 

In [340]:
array = ['Vietnamese Restaurant','Thai Restaurant','Restaurant','Japanese Restaurant','Sushi Restaurant','Malay Restaurant','Korean Restaurant','Indian Restaurant','Chinese Restaurant','Asian Restaurant']
venue_asian = nearby_venues.loc[nearby_venues['categories'].isin(array)]
venue_asian

,name,categories,lat,lng,distance,address
13,Kinn Thai,Thai Restaurant,-35.277788,149.132608,496,148 Bunda St
20,Akiba,Restaurant,-35.277612,149.131588,461,40 Bunda Street
24,Iori Japanese Restaurant,Japanese Restaurant,-35.278899,149.130554,291,41 East Row
44,Lazy Su,Asian Restaurant,-35.274680,149.132330,787,1/9 Lonsdale St
45,Blu Ginger,Indian Restaurant,-35.277346,149.131962,504,5 Genge Street
46,Lemon Grass Thai Restaurant,Thai Restaurant,-35.279319,149.128070,239,65 London Cct
52,Bistro Nguyen's,Vietnamese Restaurant,-35.278483,149.128870,311,1/80 Alinga St
69,Uni Sushi Bar,Sushi Restaurant,-35.279910,149.125750,346,22-28 University Avenue
73,Hanaichi,Japanese Restaurant,-35.280317,149.133187,378,City Wlk
79,Courgette,Restaurant,-35.276297,149.126453,606,54 Marcus Clarke St


In [372]:
# recreate the Canberra map to be used to add all asian restaurant  
map_canberra_asian = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, name, categories in zip(venue_asian['lat'], venue_asian['lng'], venue_asian['name'], venue_asian['categories']):
    label = '{}, {}'.format(name, categories)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_canberra_asian)  

map_canberra_asian

# make the data ready for K-means 

In [341]:
venue_asian.groupby('categories').count()

,name,lat,lng,distance,address
categories,,,,,
Asian Restaurant,2,2,2,2,2
Indian Restaurant,1,1,1,1,1
Japanese Restaurant,2,2,2,2,2
Korean Restaurant,1,1,1,1,1
Restaurant,2,2,2,2,2
Sushi Restaurant,1,1,1,1,1
Thai Restaurant,2,2,2,2,2
Vietnamese Restaurant,1,1,1,1,1


In [342]:
# count the number of Asian restaurant
venue_asian.shape

(12, 6)

# clean the data and select all restaurants 

In [343]:
venue_resturant = nearby_venues[nearby_venues.categories.str.contains('Restaurant')]
venue_resturant.shape

(22, 6)

In [373]:
# recreate the Canberra map to be used to add all asian restaurant  
map_canberra_resturant = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, name, categories in zip(venue_resturant['lat'], venue_resturant['lng'], venue_resturant['name'], venue_resturant['categories']):
    label = '{}, {}'.format(name, categories)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_canberra_resturant)  

map_canberra_resturant

In [344]:
# print the restaurant dataframe 
venue_resturant

,name,categories,lat,lng,distance,address
1,Mezzalira Ristorante,Italian Restaurant,-35.279308,149.127748,254,West Row
13,Kinn Thai,Thai Restaurant,-35.277788,149.132608,496,148 Bunda St
20,Akiba,Restaurant,-35.277612,149.131588,461,40 Bunda Street
24,Iori Japanese Restaurant,Japanese Restaurant,-35.278899,149.130554,291,41 East Row
26,Temporada,Tapas Restaurant,-35.276089,149.128254,582,15 Moore St.
38,Banana Leaf Restaurant,Gluten-free Restaurant,-35.281020,149.132640,315,240-250 City Walk
44,Lazy Su,Asian Restaurant,-35.274680,149.132330,787,1/9 Lonsdale St
45,Blu Ginger,Indian Restaurant,-35.277346,149.131962,504,5 Genge Street
46,Lemon Grass Thai Restaurant,Thai Restaurant,-35.279319,149.128070,239,65 London Cct
47,Sage Restaurant,Molecular Gastronomy Restaurant,-35.277729,149.137592,859,"Gorman House, Batman St."


## Grouping rows by categories 

In [345]:
#venue_grouped_all = venue_resturant.groupby('categories').mean().reset_index()
#sanity check 
venue_grouped_all = venue_resturant.groupby('categories').count().reset_index()
venue_grouped_all

,categories,name,lat,lng,distance,address
0,Asian Restaurant,2,2,2,2,2
1,Fast Food Restaurant,1,1,1,1,1
2,Gluten-free Restaurant,1,1,1,1,1
3,Indian Restaurant,1,1,1,1,1
4,Italian Restaurant,4,4,4,4,4
5,Japanese Restaurant,2,2,2,2,2
6,Korean Restaurant,1,1,1,1,1
7,Mediterranean Restaurant,1,1,1,1,1
8,Molecular Gastronomy Restaurant,1,1,1,1,1
9,Restaurant,2,2,2,2,2


In [346]:
venue_grouped_asian = venue_asian.groupby('categories').mean().reset_index()
#sanity check 
venue_grouped_asian

,categories,lat,lng,distance
0,Asian Restaurant,-35.276177,149.132380,641.5
1,Indian Restaurant,-35.277346,149.131962,504.0
2,Japanese Restaurant,-35.279608,149.131871,334.5
3,Korean Restaurant,-35.278076,149.132183,447.0
4,Restaurant,-35.276954,149.129021,533.5
5,Sushi Restaurant,-35.279910,149.125750,346.0
6,Thai Restaurant,-35.278553,149.130339,367.5
7,Vietnamese Restaurant,-35.278483,149.128870,311.0


# Run k-means to cluster the neighborhood into 3 clusters for all resturants.

In [347]:
venue_grouped_all.shape[0]

15

In [348]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [359]:
import pandas as pd
import numpy as np
# create a new dataframe
num_top_venues = venue_grouped_all.shape
columns = ['categories']
venues_sorted = pd.DataFrame(columns=columns)
venues_sorted['categories'] = venue_grouped_all['categories']

#for ind in np.arange(venue_grouped_all.shape[0]):
#    venues_sorted.['categories'] = venue_grouped_all.['categories']

venues_sorted['categories'] = venue_grouped_all['categories']

#venues_sorted
venues_sorted

,categories
0,Asian Restaurant
1,Fast Food Restaurant
2,Gluten-free Restaurant
3,Indian Restaurant
4,Italian Restaurant
5,Japanese Restaurant
6,Korean Restaurant
7,Mediterranean Restaurant
8,Molecular Gastronomy Restaurant
9,Restaurant


In [350]:
# set number of clusters
kclusters = 3

venue_grouped_all_clustering = venue_grouped_all.drop('categories', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(venue_grouped_all_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([1, 0, 0, 0, 2, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0], dtype=int32)

In [351]:
# add clustering labels
venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
venue_merged = venue_grouped_all

# merge grouped with data to add latitude/longitude for each neighborhood
venue_merged = venue_merged.join(venues_sorted.set_index('categories'), on='categories')

In [352]:
venue_merged

,categories,name,lat,lng,distance,address,Cluster Labels
0,Asian Restaurant,2,2,2,2,2,1
1,Fast Food Restaurant,1,1,1,1,1,0
2,Gluten-free Restaurant,1,1,1,1,1,0
3,Indian Restaurant,1,1,1,1,1,0
4,Italian Restaurant,4,4,4,4,4,2
5,Japanese Restaurant,2,2,2,2,2,1
6,Korean Restaurant,1,1,1,1,1,0
7,Mediterranean Restaurant,1,1,1,1,1,0
8,Molecular Gastronomy Restaurant,1,1,1,1,1,0
9,Restaurant,2,2,2,2,2,1


# identify Clustring member 

In [353]:
# Group 0 
venue_grouped_all.loc[venue_merged['Cluster Labels'] == 0]

,categories,name,lat,lng,distance,address
1,Fast Food Restaurant,1,1,1,1,1
2,Gluten-free Restaurant,1,1,1,1,1
3,Indian Restaurant,1,1,1,1,1
6,Korean Restaurant,1,1,1,1,1
7,Mediterranean Restaurant,1,1,1,1,1
8,Molecular Gastronomy Restaurant,1,1,1,1,1
10,Sushi Restaurant,1,1,1,1,1
11,Szechuan Restaurant,1,1,1,1,1
12,Tapas Restaurant,1,1,1,1,1
14,Vietnamese Restaurant,1,1,1,1,1


In [354]:
# Group 1
venue_grouped_all.loc[venue_merged['Cluster Labels'] == 1]

,categories,name,lat,lng,distance,address
0,Asian Restaurant,2,2,2,2,2
5,Japanese Restaurant,2,2,2,2,2
9,Restaurant,2,2,2,2,2
13,Thai Restaurant,2,2,2,2,2


In [355]:
# Group 2
venue_grouped_all.loc[venue_merged['Cluster Labels'] == 2]

,categories,name,lat,lng,distance,address
4,Italian Restaurant,4,4,4,4,4


In [356]:
# Group 3
#venue_grouped_all.loc[venue_merged['Cluster Labels'] == 3]

,categories,name,lat,lng,distance,address


# Run k-means to cluster the neighborhood into 3 clusters for asian resturants.

In [357]:
venue_grouped_asian.shape[0]

8

In [358]:
# create a new dataframe
num_top_venues = venue_grouped_all.shape
columns = ['categories']
venues_sorted_asian = pd.DataFrame(columns=columns)
venues_sorted_asian['categories'] = venue_grouped_asian['categories']

venues_sorted_asian['categories'] = venue_grouped_asian['categories']

#venues_sorted
venues_sorted_asian

,categories
0,Asian Restaurant
1,Indian Restaurant
2,Japanese Restaurant
3,Korean Restaurant
4,Restaurant
5,Sushi Restaurant
6,Thai Restaurant
7,Vietnamese Restaurant


In [360]:
# set number of clusters
kclusters = 3

venue_grouped_asian_clustering = venue_grouped_asian.drop('categories', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(venue_grouped_asian_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([2, 0, 1, 0, 0, 1, 1, 1], dtype=int32)

In [361]:
# add clustering labels
venues_sorted_asian.insert(0, 'Cluster Labels', kmeans.labels_)
venue_merged_asian = venue_grouped_asian

# merge grouped with data to add latitude/longitude for each neighborhood
venue_merged_asian = venue_merged_asian.join(venues_sorted_asian.set_index('categories'), on='categories')

In [362]:
venue_merged_asian

,categories,lat,lng,distance,Cluster Labels
0,Asian Restaurant,-35.276177,149.132380,641.5,2
1,Indian Restaurant,-35.277346,149.131962,504.0,0
2,Japanese Restaurant,-35.279608,149.131871,334.5,1
3,Korean Restaurant,-35.278076,149.132183,447.0,0
4,Restaurant,-35.276954,149.129021,533.5,0
5,Sushi Restaurant,-35.279910,149.125750,346.0,1
6,Thai Restaurant,-35.278553,149.130339,367.5,1
7,Vietnamese Restaurant,-35.278483,149.128870,311.0,1


## identify Clustring member

In [363]:
# Group 0
venue_grouped_asian.loc[venue_merged_asian['Cluster Labels'] == 0]

,categories,lat,lng,distance
1,Indian Restaurant,-35.277346,149.131962,504.0
3,Korean Restaurant,-35.278076,149.132183,447.0
4,Restaurant,-35.276954,149.129021,533.5


In [364]:
# Group 1
venue_grouped_asian.loc[venue_merged_asian['Cluster Labels'] == 1]

,categories,lat,lng,distance
2,Japanese Restaurant,-35.279608,149.131871,334.5
5,Sushi Restaurant,-35.279910,149.125750,346.0
6,Thai Restaurant,-35.278553,149.130339,367.5
7,Vietnamese Restaurant,-35.278483,149.128870,311.0


In [365]:
# Group 2
venue_grouped_asian.loc[venue_merged_asian['Cluster Labels'] == 2]

,categories,lat,lng,distance
0,Asian Restaurant,-35.276177,149.13238,641.5


# Conclusion 
 Based on the out put in both clustrs I suggestion to open an Asian resturant in Canberra in Braddon  
 For more information please refer to the reprot 